In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from skimage.transform import resize 

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename) 
    if (ds.BodyPartExamined !='CHEST'):
        return None
    if (ds.Modality !='DX'):
        return None
    if (ds.PatientPosition !='PA') & (ds.PatientPosition != 'AP'):
        return None
    
    img = ds.pixel_array
    return img
    

In [3]:
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input

def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    img = img/255
    IMG_SIZE = (1,224,224,3)
    proc_img = (img-img_mean)/img_std
    proc_img=resize(img, IMG_SIZE)
    
    return proc_img
    

In [4]:
# This function loads in our trained model w/ weights and compiles it 
from keras.models import model_from_json

def load_model(model_path, weight_path):

    json_file = open(model_path, "r") 
    loaded_model_json = json_file.read() 
    json_file.close() 
    my_model = model_from_json(loaded_model_json) 
    my_model.load_weights(weight_path)
    print("Loaded model from disk")
    
    return my_model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(my_model, img, thresh): 

    # todo    
    result = my_model.predict(img)
    predict=result[0]
    if (predict < thresh):
        prediction = 'No pneumonia'
    if(predict > thresh):
        prediction = 'Pneumonia'
    
    return prediction 

In [5]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']
model_path = 'my_model.json'
weight_path = r'xray_class_my_model.best.hdf5'

 # This might be different if you did not use vgg16



my_model=load_model(model_path, weight_path)
thresh=0.38

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
    img_reshape = img.reshape((img.shape[0], img.shape[1]))
    img_mean = np.mean(img)
    img_std = np.std(img)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    IMG_SIZE=(1,224,224,3)
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc, thresh)
    print(pred)

Loaded model from disk
Load file test1.dcm ...
No pneumonia
Load file test2.dcm ...
No pneumonia
Load file test3.dcm ...
Pneumonia
Load file test4.dcm ...
Load file test5.dcm ...
Load file test6.dcm ...


In [10]:
dcm1=pydicom.dcmread(test_dicoms[0])
dcm1

(0008, 0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.11129.5.5.110503645592756492463169821050252582267888
(0008, 0060) Modality                            CS: 'DX'
(0008, 1030) Study Description                   LO: 'No Finding'
(0010, 0020) Patient ID                          LO: '2'
(0010, 0040) Patient's Sex                       CS: 'M'
(0010, 1010) Patient's Age                       AS: '81'
(0018, 0015) Body Part Examined                  CS: 'CHEST'
(0018, 5100) Patient Position                    CS: 'PA'
(0020, 000d) Study Instance UID                  UI: 1.3.6.1.4.1.11129.5.5.112507010803284478207522016832191866964708
(0020, 000e) Series Instance UID                 UI: 1.3.6.1.4.1.11129.5.5.112630850362182468372440828755218293352329
(0028, 0002) Samples per Pixel                   US: 1
(0028, 0004) Photometric Interpretation          CS: 'MONOCHROME2'
(0028, 0010) Rows       

In [8]:
dcm2=pydicom.dcmread(test_dicoms[1])
dcm2

(0008, 0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.11129.5.5.153751009835107614666834563294684339746480
(0008, 0060) Modality                            CS: 'DX'
(0008, 1030) Study Description                   LO: 'Cardiomegaly'
(0010, 0020) Patient ID                          LO: '1'
(0010, 0040) Patient's Sex                       CS: 'M'
(0010, 1010) Patient's Age                       AS: '58'
(0018, 0015) Body Part Examined                  CS: 'CHEST'
(0018, 5100) Patient Position                    CS: 'AP'
(0020, 000d) Study Instance UID                  UI: 1.3.6.1.4.1.11129.5.5.111396399361969898205364400549799252857604
(0020, 000e) Series Instance UID                 UI: 1.3.6.1.4.1.11129.5.5.195628213694300498946760767481291263511724
(0028, 0002) Samples per Pixel                   US: 1
(0028, 0004) Photometric Interpretation          CS: 'MONOCHROME2'
(0028, 0010) Rows     

In [9]:
dcm3=pydicom.dcmread(test_dicoms[2])
dcm3

(0008, 0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.11129.5.5.179222148351666120521423991179194552820263
(0008, 0060) Modality                            CS: 'DX'
(0008, 1030) Study Description                   LO: 'Effusion'
(0010, 0020) Patient ID                          LO: '61'
(0010, 0040) Patient's Sex                       CS: 'M'
(0010, 1010) Patient's Age                       AS: '77'
(0018, 0015) Body Part Examined                  CS: 'CHEST'
(0018, 5100) Patient Position                    CS: 'AP'
(0020, 000d) Study Instance UID                  UI: 1.3.6.1.4.1.11129.5.5.189886800072183603320722059194952488628637
(0020, 000e) Series Instance UID                 UI: 1.3.6.1.4.1.11129.5.5.110145974268321300517474523922373370343198
(0028, 0002) Samples per Pixel                   US: 1
(0028, 0004) Photometric Interpretation          CS: 'MONOCHROME2'
(0028, 0010) Rows        

In [2]:
model_path.summary()

NameError: name 'model_path' is not defined